In [1]:
import tensorflow as tf

class MADModel(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec([4, None], tf.int32)])
    def __call__(self, inputs):
        # inputs: shape [4, N], dtype int32
        timestamps = inputs[0]          # row 0: timestamps em ms
        x = tf.cast(inputs[1], tf.float32)
        y = tf.cast(inputs[2], tf.float32)
        z = tf.cast(inputs[3], tf.float32)
        # Magnitude vetorial
        mag = tf.sqrt(x*x + y*y + z*z)
        # Índices de janela (cada 5s = 5000 ms)
        win = tf.cast(tf.floor(tf.cast(timestamps, tf.float32) / 5000.0), tf.int32)
        # Ajusta para começar do zero (opcional)
        min_win = tf.reduce_min(win)
        win0 = win - min_win
        # Soma e contagem por janela
        num_wins = tf.reduce_max(win0) + 1
        sums = tf.math.unsorted_segment_sum(mag, win0, num_wins)
        counts = tf.math.unsorted_segment_sum(tf.ones_like(mag), win0, num_wins)
        means = sums / counts  # pode gerar NaN onde count=0
        valid = counts > 0
        # Número de janelas não vazias
        num_valid = tf.reduce_sum(tf.cast(valid, tf.float32))
        # Soma dos meios de janela válidos e soma de seus quadrados
        sum_means = tf.reduce_sum(tf.where(valid, means, tf.constant(0.0)))
        sum_sq = tf.reduce_sum(tf.where(valid, means*means, tf.constant(0.0)))
        # Estatísticas finais
        mean_of_means = sum_means / num_valid
        mean_sq = sum_sq / num_valid
        std_dev = tf.sqrt(mean_sq - mean_of_means*mean_of_means)
        # mínimo e máximo ignorando slots inválidos
        safe_min = tf.where(valid, means, tf.constant(1e30))
        safe_max = tf.where(valid, means, tf.constant(-1e30))
        min_val = tf.reduce_min(safe_min)
        max_val = tf.reduce_max(safe_max)
        # Empacota saída [mean, std, min, max]
        return tf.stack([mean_of_means, std_dev, min_val, max_val])

/Users/joaovitorcotta/StudioProjects/VulkanFFT/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import os

# Caminho do modelo
tflite_path = "mad_model.tflite"

# Remove o modelo anterior, se existir
if os.path.exists(tflite_path):
    os.remove(tflite_path)
    print("Modelo antigo removido.")

# Cria o modelo
mad_model = MADModel()

# Define a função concreta explicitamente com shape dinâmico
concrete_fn = mad_model.__call__.get_concrete_function(
    tf.TensorSpec(shape=[4, None], dtype=tf.int32)
)

# Converte para TFLite com suporte a operadores do TF
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [concrete_fn],
    mad_model
)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

# Salva o modelo
with open(tflite_path, "wb") as f:
    f.write(tflite_model)
    print("Novo modelo salvo com sucesso.")

# Verifica se o modelo foi salvo corretamente
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()

# ✅ Esperado: shape [4, -1]
print("Shape de entrada:", input_details[0]['shape'])
print("Shape signature :", input_details[0]['shape_signature'])
print("Tipo de entrada :", input_details[0]['dtype'])

INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmpu8ip6n3e/assets


INFO:tensorflow:Assets written to: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmpu8ip6n3e/assets


Novo modelo salvo com sucesso.
Shape de entrada: [4 1]
Shape signature : [ 4 -1]
Tipo de entrada : <class 'numpy.int32'>


W0000 00:00:1750712531.828495 22855745 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750712531.828515 22855745 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-23 18:02:11.829008: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmpu8ip6n3e
2025-06-23 18:02:11.829190: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-23 18:02:11.829194: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/qv/p5h208kn6gd462qyk1_qdbh00000gn/T/tmpu8ip6n3e
I0000 00:00:1750712531.830538 22855745 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-06-23 18:02:11.830710: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-23 18:02:11.835012: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/fol

In [4]:
import numpy as np
import tensorflow as tf
import time
import math

# Função equivalente à sua getMAD() em Python
def get_mad_py(data):
    timestamps = data[0]
    magnitudes = np.sqrt(np.sum(np.square(data[1:]), axis=0))
    magnitude_by_time = list(zip(timestamps, magnitudes))

    window_size_ms = 5000  # 5 segundos
    start_time = magnitude_by_time[0][0]
    end_time = magnitude_by_time[-1][0]

    block_avgs = []
    current_start = start_time

    while current_start <= end_time:
        current_end = current_start + window_size_ms
        values = [mag for ts, mag in magnitude_by_time if current_start <= ts < current_end]
        if values:
            block_avgs.append(np.mean(values))
        current_start = current_end

    mean = np.mean(block_avgs)
    std_dev = np.std(block_avgs)
    min_val = np.min(block_avgs)
    max_val = np.max(block_avgs)

    return np.array([mean, std_dev, min_val, max_val], dtype=np.float32)

# Geração de dados simulados
N = 10000
timestamps = np.arange(N, dtype=np.int32) * 20
x = np.random.randint(0, 10, N, dtype=np.int32)
y = np.random.randint(0, 10, N, dtype=np.int32)
z = np.random.randint(0, 10, N, dtype=np.int32)
input_data = np.stack([timestamps, x, y, z], axis=0)  # shape [4, N]

# --- TEMPO: função Python pura ---
start_cpu = time.time()
mad_ref = get_mad_py(input_data)
end_cpu = time.time()
cpu_time_ms = (end_cpu - start_cpu) * 1000

# --- TEMPO: carregamento do modelo TFLite ---
start_load = time.time()
interpreter = tf.lite.Interpreter(model_path="mad_model.tflite")
end_load = time.time()
load_time_ms = (end_load - start_load) * 1000

# Redimensiona entrada e aloca tensores
N = input_data.shape[1]
interpreter.resize_tensor_input(0, [4, N])
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# --- TEMPO: execução do modelo TFLite ---
start_tflite = time.time()
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
mad_out = interpreter.get_tensor(output_details[0]['index'])[0]
end_tflite = time.time()
tflite_inference_time_ms = (end_tflite - start_tflite) * 1000

# --- Cálculo da redução de tempo ---
reduction_pct = ((cpu_time_ms - tflite_inference_time_ms) / cpu_time_ms) * 100

# --- Resultados ---
print("Referência (Python): ", mad_ref)
print("Modelo TFLite      : ", mad_out)
print("Erro absoluto      : ", np.abs(mad_ref - mad_out))

print(f"\n⏱️ Tempo CPU (Python):        {cpu_time_ms:.3f} ms")
print(f"📦 Tempo de carga TFLite:      {load_time_ms:.3f} ms")
print(f"⚡ Tempo de inferência TFLite: {tflite_inference_time_ms:.3f} ms")
print(f"📉 Redução de tempo (inference): {reduction_pct:.2f}%")



Referência (Python):  [8.808858  0.1548964 8.493439  9.178185 ]
Modelo TFLite      :  8.808857
Erro absoluto      :  [9.5367432e-07 8.6539602e+00 3.1541824e-01 3.6932850e-01]

⏱️ Tempo CPU (Python):        48.635 ms
📦 Tempo de carga TFLite:      1.834 ms
⚡ Tempo de inferência TFLite: 0.227 ms
📉 Redução de tempo (inference): 99.53%
